### USC ID  : 8534717045
### NAME   : SRINATH BEGUDEM
### GITHUB: SrinathBegudem

In [1]:
import os
import pandas as pd
from scipy import stats
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import matplotlib.pyplot as plt
import statsmodels.api as sm_api
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import multilabel_confusion_matrix
from itertools import cycle
from scipy import interp
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import cross_val_score
import statistics

import warnings
warnings.filterwarnings("ignore")

1.Time Series Classification Part 1: Feature Creation/Extraction
An interesting task in machine learning is classification of time series. In this problem,
we will classify the activities of humans based on time series obtained by a Wireless
Sensor Network.
* (a) Download the AReM data from:
https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\
%29. The dataset contains 7 folders that represent seven types of activities. In
each folder, there are multiple files each of which represents an instant of a human
performing an activity.
1
Each file containis 6 time series collected from activities
of the same person, which are called avg
rss12, var
rss12, avg
rss13, var
rss13,
vg
rss23, and ar
rss23. There are 88 instances in the dataset, each of which con-
tains 6 time series and each time series has 480 consecutive values.
* b)
Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,
2, and 3 in other folders as test data and other datasets as train data

In [2]:
def load_data_from_folder(foldername, start, end):
    file_paths = []

    for i in range(start, end + 1):
        file_path = f"{foldername}\\dataset{i}.csv"
        file_paths.append(file_path)

    data_frames = []

    for file_path in file_paths:
        df = pd.read_csv(
            file_path,
            skiprows=5,
            on_bad_lines="skip",
            header=None,
            index_col=None,
            names=['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
        )
        data_frames.append(df)

    concatenated_df = pd.concat(data_frames, ignore_index=True)

    total_datasets = end - start + 1

    return concatenated_df, total_datasets

root_dir = '..\AReM'


bending1_test_data, bending1_test_total = load_data_from_folder(os.path.join(root_dir, 'bending1'), 1, 2)
bending2_test_data, bending2_test_total = load_data_from_folder(os.path.join(root_dir, 'bending2'), 1, 2)
cycling_test_data, cycling_test_total = load_data_from_folder(os.path.join(root_dir, 'cycling'), 1, 3)
lying_test_data, lying_test_total = load_data_from_folder(os.path.join(root_dir, 'lying'), 1, 3)
sitting_test_data, sitting_test_total = load_data_from_folder(os.path.join(root_dir, 'sitting'), 1, 3)
standing_test_data, standing_test_total = load_data_from_folder(os.path.join(root_dir, 'standing'), 1, 3)
walking_test_data, walking_test_total = load_data_from_folder(os.path.join(root_dir, 'walking'), 1, 3)

bending1_train_data, bending1_train_total = load_data_from_folder(os.path.join(root_dir, 'bending1'), 3, 7)
bending2_train_data, bending2_train_total = load_data_from_folder(os.path.join(root_dir, 'bending2'), 3, 6)
cycling_train_data, cycling_train_total = load_data_from_folder(os.path.join(root_dir, 'cycling'), 4, 15)
lying_train_data, lying_train_total = load_data_from_folder(os.path.join(root_dir, 'lying'), 4, 15)
sitting_train_data, sitting_train_total = load_data_from_folder(os.path.join(root_dir, 'sitting'), 4, 15)
standing_train_data, standing_train_total = load_data_from_folder(os.path.join(root_dir, 'standing'), 4, 15)
walking_train_data, walking_train_total = load_data_from_folder(os.path.join(root_dir, 'walking'), 4, 15)

combined_test_data = pd.concat([
    bending1_test_data,
    bending2_test_data,
    cycling_test_data,
    lying_test_data,
    sitting_test_data,
    standing_test_data,
    walking_test_data
])

combined_train_data = pd.concat([
    bending1_train_data,
    bending2_train_data,
    cycling_train_data,
    lying_train_data,
    sitting_train_data,
    standing_train_data,
    walking_train_data
])

total_test = sum([
    bending1_test_total,
    bending2_test_total,
    cycling_test_total,
    lying_test_total,
    sitting_test_total,
    standing_test_total,
    walking_test_total
])

total_train = sum([
    bending1_train_total,
    bending2_train_total,
    cycling_train_total,
    lying_train_total,
    sitting_train_total,
    standing_train_total,
    walking_train_total
])

print(f"Total datasets in test data: {total_test}")
print(f"Total datasets in train data: {total_train}")


Total datasets in test data: 19
Total datasets in train data: 69


In [3]:
combined_test_data

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.30
1,250,39.25,0.43,23.00,0.00,33.00,0.00
2,500,39.25,0.43,23.25,0.43,33.00,0.00
3,750,39.50,0.50,23.00,0.71,33.00,0.00
4,1000,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...,...
1435,118750,36.00,2.45,17.00,5.10,20.50,0.87
1436,119000,34.33,1.89,15.00,2.45,17.00,2.12
1437,119250,33.00,7.35,14.60,3.14,13.00,5.70
1438,119500,31.67,1.25,11.00,6.16,19.25,2.17


In [4]:
combined_train_data

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.71,21.25,0.43,30.00,0.00
1,250,41.50,0.50,20.25,1.48,31.25,1.09
2,500,41.50,0.50,14.25,1.92,33.00,0.00
3,750,40.75,0.83,15.75,0.43,33.00,0.00
4,1000,40.00,0.71,20.00,2.74,32.75,0.43
...,...,...,...,...,...,...,...
5755,118750,34.50,6.18,9.00,3.56,12.67,4.19
5756,119000,25.75,6.02,13.75,2.05,16.00,1.58
5757,119250,31.50,3.35,10.25,5.12,16.25,2.95
5758,119500,33.75,2.77,14.00,3.24,13.75,0.43


c)
Feature Extraction
Classification of time series usually needs extracting features from them. In this
problem, we focus on time-domain features.
* i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

#### Solution
- Mean
- Median
- Standard Deviation
- Variance
- Minimum
- Maximum
- Range
- Skewness
- Kurtosis
- Entropy
- stationarity
- scaling properties
- 1st quartile
- 3rd quartile
- Interquartile Range (IQR)
- Mean Absolute Deviation (MAD)
- Coefficient of Variation (CV)
- Hurst Exponent

ii) Extract the time-domain features minimum, maximum, mean, median, stan-
dard deviation, first quartile, and third quartile for all of the 6 time series
in each instance. You are free to normalize/standardize features or use them
directly.


In [5]:
import csv
def calculate_features(instance_df):
    features = []
    
    for column in instance_df.columns:
        if pd.api.types.is_numeric_dtype(instance_df[column]):
            feature_values = instance_df[column]
            
            min_val = np.min(feature_values)
            max_val = np.max(feature_values)
            mean_val = np.mean(feature_values)
            median_val = np.median(feature_values)
            std_val = np.std(feature_values)
            q1_val = np.percentile(feature_values, 25)
            q3_val = np.percentile(feature_values, 75)
            
            features.extend([min_val, max_val, mean_val, median_val, std_val, q1_val, q3_val])
    
    return features

root_dir = '..\AReM'

columns = range(1, 100)
feat_list = []

for root, dirs, files in os.walk(root_dir):
    for dataset_file in files:
        if dataset_file.endswith('.csv'):
            dataset_file_path = os.path.join(root, dataset_file)
            
            with open(dataset_file_path, 'r') as file:
                sniffer = csv.Sniffer()
                sep = sniffer.sniff(file.read()).delimiter
                file.seek(0)
                
                if sep == ",":
                    instance_df = pd.read_csv(
                        file,
                        skiprows=5,
                        header=None,
                        on_bad_lines="skip",
                        index_col=None
                    )
                else:
                    instance_df = pd.read_csv(
                        file,
                        skiprows=5,
                        header=None,
                        sep="\s+",
                        on_bad_lines="skip",
                        index_col=None
                    )
            
            instance_features = calculate_features(instance_df)
            
            feat_list.append(instance_features)

feature_columns = []
for col in instance_df.columns:
    if pd.api.types.is_numeric_dtype(instance_df[col]):
        feature_columns.extend([f"{col}_min", f"{col}_max", f"{col}_mean", f"{col}_median", f"{col}_std", f"{col}_q1", f"{col}_q3"])

features_df = pd.DataFrame(feat_list, columns=feature_columns)
features_df = features_df.loc[:, ~features_df.columns.str.startswith('0_')]

display(features_df)


,1_min,1_max,1_mean,1_median,1_std,1_q1,1_q3,2_min,2_max,2_mean,...,5_std,5_q1,5_q3,6_min,6_max,6_mean,6_median,6_std,6_q1,6_q3
0,37.25,45.00,40.624792,40.500,1.475428,39.25,42.0000,0.0,1.30,0.358604,...,2.186168,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582308,0.00,1.3000
1,38.00,45.67,42.812812,42.500,1.434054,42.00,43.6700,0.0,1.22,0.372437,...,1.993175,32.0000,34.50,0.0,3.11,0.571083,0.430,0.600383,0.00,1.3000
2,35.00,47.40,43.954500,44.330,1.557210,43.00,45.0000,0.0,1.70,0.426250,...,1.997520,35.3625,36.50,0.0,1.79,0.493292,0.430,0.512971,0.00,0.9400
3,33.00,47.75,42.179812,43.500,3.666840,39.15,45.0000,0.0,3.00,0.696042,...,3.845436,30.4575,36.33,0.0,2.18,0.613521,0.500,0.523771,0.00,1.0000
4,33.00,45.75,41.678063,41.750,2.241152,41.33,42.7500,0.0,2.83,0.535979,...,2.408514,28.4575,31.25,0.0,1.79,0.383292,0.430,0.388759,0.00,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,20.75,46.25,34.763333,35.290,4.737266,31.67,38.2500,0.0,12.68,4.223792,...,3.171372,14.2500,18.33,0.0,9.39,3.288271,3.270,1.645811,2.05,4.3050
84,21.50,51.00,34.935812,35.500,4.641102,32.00,38.0625,0.0,12.21,4.115750,...,3.188731,14.2375,18.25,0.0,10.21,3.280021,3.015,1.699145,2.12,4.5000
85,18.33,47.67,34.333042,34.750,4.943612,31.25,38.0000,0.0,12.48,4.396958,...,2.997366,13.7500,18.00,0.0,8.01,3.261583,2.980,1.615604,2.05,4.3200
86,18.33,45.75,34.599875,35.125,4.726858,31.50,38.0000,0.0,15.37,4.398833,...,2.902659,14.0000,18.25,0.0,8.86,3.289542,3.015,1.678418,2.12,4.2600


In [6]:
features_df.std()

1_min       9.569975
1_max       4.394362
1_mean      5.335700
1_median    5.440054
1_std       1.770338
1_q1        6.153874
1_q3        5.138925
2_min       0.000000
2_max       5.062729
2_mean      1.574198
2_median    1.412293
2_std       0.883215
2_q1        0.946386
2_q3        2.125399
3_min       2.956462
3_max       4.875137
3_mean      4.008228
3_median    4.036396
3_std       0.945683
3_q1        4.220658
3_q3        4.171628
4_min       0.000000
4_max       2.183625
4_mean      1.166178
4_median    1.145985
4_std       0.457805
4_q1        0.843405
4_q3        1.552504
5_min       6.124001
5_max       5.741238
5_mean      5.675543
5_median    5.813782
5_std       1.023850
5_q1        6.096465
5_q3        5.531720
6_min       0.045838
6_max       2.518921
6_mean      1.154889
6_median    1.086474
6_std       0.517112
6_q1        0.758687
6_q3        1.523739
dtype: float64

iii.
Estimate the standard deviation of each of the time-domain features you
extracted from the data. Then, use Python’s bootstrapped or any other
method to build a 90% bootsrap confidence interval for the standard deviation
of each feature.

In [7]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

def compute_bootstrap_intervals(dataset, iterations, percentile_low, percentile_high):
    bootstrap_std_devs = []


    for _ in range(iterations):
        bootstrap_sample = resample(dataset, replace=True, n_samples=len(dataset))
        bootstrap_std = bootstrap_sample.std()
        bootstrap_std_devs.append(bootstrap_std)

   
    std_devs_df = pd.DataFrame(bootstrap_std_devs)

    
    lower_bounds = std_devs_df.apply(lambda x: np.percentile(x, percentile_low), axis=0)
    upper_bounds = std_devs_df.apply(lambda x: np.percentile(x, percentile_high), axis=0)

    
    confidence_interval_df = pd.DataFrame({
        'Lower Bound': lower_bounds,
        'Upper Bound': upper_bounds
    })

    return confidence_interval_df

bootstrap_confidence_intervals = compute_bootstrap_intervals(features_df, 1000, 5, 95)
display(bootstrap_confidence_intervals)


,Lower Bound,Upper Bound
1_min,8.235149,10.763278
1_max,3.318860,5.256214
1_mean,4.694834,5.858370
1_median,4.781237,6.003624
1_std,1.572356,1.932152
1_q1,5.573038,6.668727
1_q3,4.343051,5.801617
2_min,0.000000,0.000000
2_max,4.605395,5.379246
2_mean,1.394192,1.702588


### iv.Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).



**Mean:** This represents the average value of the dataset, providing a central point that indicates the general tendency of the data points. The mean is useful for understanding the overall level of the measured variable and is often used in conjunction with other metrics to provide a more complete picture of the data's characteristics.

**Maximum:** This indicates the highest value in the dataset, which can be crucial for identifying peaks or extreme values. The maximum value is particularly important in contexts where outliers or peak performance metrics are of interest, as it helps to understand the upper limits of the data.

**Third Quartile (Q3):** The third quartile represents the value below which 75% of the data falls. It provides insights into the upper range of the data distribution and can help identify higher percentile behavior.

**Standard Deviation:** This measures the dispersion or variability of the dataset. A higher standard deviation indicates that the data points are spread out over a larger range of values, which can be important for understanding the volatility or consistency of the data.

**Skewness:** This measures the asymmetry of the data distribution around its mean. Positive skew indicates a distribution with an asymmetric tail extending towards more positive values, while negative skew indicates a tail extending towards more negative values. Skewness can provide insights into the nature of the distribution of observed values, which can be crucial for certain types of statistical analysis where normal distribution is assumed.


# 2. ISLR 3.7.4
### I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β0 + β1X + β2X2 + β3X3 + ϵ. 
### (a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ϵ. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.
### (b) Answer (a) using test rather than training RSS.
### (c) Suppose that the true relationship between X and Y is not linear,but we don’t know how far it is from linear. Consider the trainingRSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.
### (d) Answer (c) using test rather than training RSS.




### Part (a): Training RSS for Linear vs. Cubic Regression with a Linear True Relationship

Given a true linear relationship between $X$ and $Y$, the training RSS for a linear regression model directly reflects the variance in the data that is not explained by the linear model. In contrast, a cubic regression ($Y = \beta_0 + \beta_1X + \beta_2X^2 + \beta_3X^3 + \epsilon$) can potentially fit the training data more closely due to its additional parameters, allowing for a lower training RSS. However, this does not imply that cubic regression is more appropriate; it merely has greater flexibility, which may lead to overfitting, especially if the true underlying relationship is indeed linear.

### Part (b): Test RSS for Linear vs. Cubic Regression with a Linear True Relationship

When evaluating the model's performance on unseen data (test RSS), the simpler linear regression model is expected to outperform the cubic regression if the true relationship is linear. This is because the cubic regression's added complexity, while potentially reducing training RSS, is likely to result in overfitting, capturing noise in the training data as if it were a true signal. Consequently, the linear model, being more aligned with the true linear relationship, is expected to generalize better, leading to a lower test RSS.

### Part (c): Training RSS for Linear vs. Cubic Regression with a Nonlinear True Relationship

If the true relationship between $X$ and $Y$ is nonlinear, the training RSS comparison between linear and cubic regression models becomes contingent on the nature of the nonlinearity. The cubic regression's flexibility enables it to model complex relationships more effectively, likely resulting in a lower training RSS compared to the linear model. This assumes that the nonlinearity in the true relationship can be better captured by the cubic model's additional terms.

### Part (d): Test RSS for Linear vs. Cubic Regression with a Nonlinear True Relationship

For a nonlinear true relationship, the test RSS will critically depend on the model's ability to generalize beyond the training data. The cubic regression may offer a better fit for the training data and, if the nonlinearity it models is reflective of the true relationship, could potentially lead to a lower test RSS. However, the risk of overfitting remains significant; if the cubic model captures noise as signal, its performance on new data may degrade. Conversely, a linear model, despite possibly having a higher bias for assuming linearity, might exhibit lower variance and better generalization to new data, particularly if the true relationship's deviation from linearity is modest.

In essence, the choice between a linear and cubic model for both training and test RSS should be informed by the underlying relationship's complexity and the models' respective bias-variance trade-offs.
